In [1]:
import warnings
warnings.filterwarnings("ignore")

from utils import dataset, load_bow
import pandas as pd
from datetime import datetime
from itertools import chain
import numpy as np

In [2]:
w = 1

In [3]:
col = [
    'Publication Type', \
    'Venue', \
    'Author Full Names', \
    'Article Title', \
    'Abstract', \
    'Publication Date', \
    'Publication Year']

venues = dataset()[col]
venues = venues.dropna(subset=['Abstract'])
venues['Abstract Lemmatized'] = load_bow('25_venues')
venues = venues.dropna(subset=['Publication Date'])
venues['Date'] = (venues['Publication Date']+'-'+venues['Publication Year']).apply(lambda x: datetime.strptime(x, '%b-%Y'))
query_dataframe = pd.read_csv('./database/query_dataframe.csv', index_col=1)
venues['Venue Full Name'] = venues['Venue'].apply(lambda x : query_dataframe.loc[x, 'Nome'])
venues = venues.sort_values(by='Date')
venues.index = range(len(venues))

In [4]:
authors = venues['Author Full Names']
authors = [names.split(';') for names in authors]
authors = [[name.strip().upper() for name in names] for names in authors]
authors = list(authors)
authors = list(chain.from_iterable(authors))
authors = sorted(list(set(authors)))

In [5]:
authors_dict = {}

for author in authors:
    authors_dict[author] = []

for index, row in venues.iterrows():
    for author in row['Author Full Names'].split(';'):
        authors_dict[author.upper().strip()].append(row['Article Title'])

In [6]:
authors_df = {k: str(v) for k, v in authors_dict.items()}
authors_df = pd.DataFrame(authors_df, index = ['Papers']).T
authors_df['publications'] = authors_df['Papers'].apply(lambda x: len(eval(x)))
authors_df.to_csv('./database/authors.csv')

In [48]:
top = pd.read_csv('./database/1000authors_pids.csv', index_col=0)
top['Papers'] = top['Papers'].apply(lambda x: eval(x))
dups = top[top.duplicated(subset=['pids'], keep=False)].sort_values('pids')
combined_rows = pd.DataFrame(columns=top.columns)

for dup in np.unique(dups['pids']):
    duplicate_rows = dups[dups['pids'] == dup]
    combined_papers = []
    for papers in duplicate_rows['Papers']:
        combined_papers += papers
    combined_row = duplicate_rows.iloc[0].copy()
    combined_row['Papers'] = combined_papers
    combined_row.name = duplicate_rows.index[0]
    combined_rows = combined_rows.append(combined_row)

top = top.drop(dups.index)
top = top.append(combined_rows)

top['publications'] = top['Papers'].apply(lambda x: len(x))

In [54]:
lenght = 1000
p = 0
top_pubs = pd.DataFrame()
while len(top_pubs) > lenght:
    top_pubs = top.iloc[np.where(top['publications'] >= p)].sort_index()
    p += 1
top_pubs = top.iloc[np.where(top['publications'] >= p-2)].sort_index()[['Papers', 'publications']]
top_pubs['pids'] = np.nan
top_pubs.to_csv(f'./database/top_authors/top{lenght}authors.csv')